In [21]:
import json
import requests
import urllib.parse
base_url = 'https://itunes.apple.com/search'

artist = 'Eminem'

params = {
    'term': artist,
    'limit': 200,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

# Все доступные ключи объекта
print(list(data[0].keys()))

['wrapperType', 'kind', 'artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionCensoredName', 'trackCensoredName', 'artistViewUrl', 'collectionViewUrl', 'trackViewUrl', 'previewUrl', 'artworkUrl30', 'artworkUrl60', 'artworkUrl100', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'contentAdvisoryRating', 'isStreamable']


In [22]:
import dateutil.parser

# Преобразование строки с датой в объект даты
# **x - скопировать все поля из объекта x
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)
data = list(data)
# Убираем лишние поля
data = map(lambda x: {
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionCensoredName'],
    'trackName': x['trackCensoredName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'collectionExplicitness': x['collectionExplicitness'],
    'trackExplicitness': x['trackExplicitness'],
    'discCount': x['discCount'],
    'discNumber': x['discNumber'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName'],
    'isStreamable': x['isStreamable']
}, data)
data = list(data)
print(list(data[0].keys()))

print(set(map(lambda x: x['artistName'], data)))


['artistId', 'collectionId', 'trackId', 'artistName', 'collectionName', 'trackName', 'collectionPrice', 'trackPrice', 'releaseDate', 'collectionExplicitness', 'trackExplicitness', 'discCount', 'discNumber', 'trackCount', 'trackNumber', 'trackTimeMillis', 'country', 'currency', 'primaryGenreName', 'isStreamable']
{'Skylar Grey, Eminem & Yelawolf', 'Eminem, 50 Cent, Ca$his & Lloyd Banks', 'Eminem', 'Eminem & 50 Cent', 'Eminem & Jessie Reyez', 'Eminem & P!nk', 'Eminem, Dr. Dre & 50 Cent', 'Drake, Kanye West, Lil Wayne & Eminem', "Eminem, Royce da 5'9, Big Sean, Danny Brown, DeJ Loaf & Trick Trick", 'Eminem & Steve Berman'}


In [23]:
print('Название исполнителя: ',artist)
print('Самые популярные жанры: ',set(map(lambda x: x['primaryGenreName'], data)))

Название исполнителя:  Eminem
Самые популярные жанры:  {'Hip-Hop/Rap', 'Soundtrack'}


In [24]:
from itertools import groupby
from re import I

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']

del sorted_by_album

I = 0
print("Количество альбомов: ")
for album_names in album_names.values():
    I = I + 1
print(I)

Количество альбомов: 
32


In [25]:
from itertools import groupby
from re import A
track_count_extractor = lambda x: x['trackCount']

sorted_by_track = sorted(data, key=track_count_extractor)
track_groups = {}
track_names = {}

for track_id, songs in groupby(sorted_by_track, key=track_count_extractor):
    songs = list(songs)
    track_groups[track_id] = songs 
    track_names[track_id] = songs[0]['trackName']


A = 0
print("Количество песен: ")
for track_names in track_names.values():
    A = A + 1
print(A)

Количество песен: 
17


In [45]:

def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}H {minutes}M {seconds}S"

time_extractor = lambda x: x['trackTimeMillis']

longest_song = max(data, key=time_extractor)
print('Самая длинная песня:', longest_song['trackName'], ' и её длительность: ', convert_millis(longest_song['trackTimeMillis']))

shortest_song = min(data, key=time_extractor)
print('Самая короткая песня:', shortest_song['trackName'], ' и её длительность: ', convert_millis(shortest_song['trackTimeMillis']))

Самая длинная песня: Bad Guy  и её длительность:  0H 7M 14S
Самая короткая песня: Paul Rosenberg (Skit)  и её длительность:  0H 0M 22S


In [28]:
import statistics
print( 'Общая длительность песен:', convert_millis( sum( list( map( lambda x: x['trackTimeMillis'], data)))))
print( 'Средняя длительность песен:', convert_millis( round( statistics.mean( list( map( lambda x : x['trackTimeMillis'], data))))))
print( 'Медиана длительности песен:', convert_millis( round( statistics.median( list( map( lambda x : x['trackTimeMillis'], data )))))) 

Общая длительность песен: 14H 42M 35S
средняя длительность песен: 0H 4M 24S
медиана длительности песен: 0H 4M 35S


In [44]:
price_extractor = lambda x: x['trackPrice']

firstprized_song = max(data, key=price_extractor)
print('Самая дорогая песня:', firstprized_song['trackName'], ' и её стоимость: ', firstprized_song['trackPrice'])

lastprized_song = min(data, key=price_extractor)
print('Самая дешёвая песня:', lastprized_song['trackName'], ' и её стоимость: ', lastprized_song['trackPrice'])

Самая дорогая песня: Love the Way You Lie (feat. Rihanna)  и её стоимость:  1.29
Самая дешёвая песня: Without Me  и её стоимость:  -1.0


In [30]:
print( 'Общая стоимость песен: ', round(sum( list( map( lambda x: x['trackPrice'], data))),2))
print( 'Средняя стоимость песен: ', round(statistics.mean( list( map( lambda x: x['trackPrice'], data))), 2))
print( 'Медиана стоимости песен: ', round(statistics.median( list( map( lambda x: x['trackPrice'], data))), 2))


Общая стоимость песен:  249.93
Средняя стоимость песен:  1.25
Медиана стоимости песен:  1.29


In [55]:
for i in range(I):
    album_name = list( set( map( lambda x: x['collectionName'], data)))[i]
    album_data = list( filter( lambda x:album_name in x['collectionName'], data))

    
    print( 'Название альбома:', set(list( map( lambda x: x['collectionName'], album_data))))
    print( 'Количество песен:', len( list( map( lambda x: x['trackName'], album_data))))
    print( 'Самый популярный жанр:',  set(list( map( lambda x: x['primaryGenreName'], album_data))))
    print( 'Дата релиза альбома:',  list( map( lambda x: x['releaseDate'], album_data))[0])
    
    longest_song = max(data, key=time_extractor)
    print('Самая длинная песня:', longest_song['trackName'], ' и её длительность: ', convert_millis(longest_song['trackTimeMillis']))
    shortest_song = min(data, key=time_extractor)
    print('Самая короткая песня:', shortest_song['trackName'], ' и её длительность: ', convert_millis(shortest_song['trackTimeMillis']))
    
    print( 'Общая длительность песен:', convert_millis( sum( list( map( lambda x: x['trackTimeMillis'], album_data)))))
    print( 'Средняя длительность песен:', convert_millis( round( statistics.mean( list( map( lambda x : x['trackTimeMillis'], album_data))))))
    print( 'Медиана длительности песен:', convert_millis( round( statistics.median( list( map( lambda x : x['trackTimeMillis'], album_data))))))
    
    
    firstprized_song = max(data, key=price_extractor)
    print('Самая дорогая песня:', firstprized_song['trackName'], ' и её стоимость: ', firstprized_song['trackPrice'])
    lastprized_song = min(data, key=price_extractor)
    print('Самая дешёвая песня:', lastprized_song['trackName'], ' и её стоимость: ', lastprized_song['trackPrice'])

    print( 'оОбщая стоимость песен: ', round( sum( list( map( lambda x: x['trackPrice'], album_data ) ) ), 2 ))
    print( 'Средняя стоимость песен: ', round( statistics.mean( list( map( lambda x: x['trackPrice'], album_data ) ) ), 2 ))
    print( 'Медиана стоимости песен: ', round( statistics.median( list( map( lambda x: x['trackPrice'], album_data ) ) ), 2 ))
    
    


Название альбома: {'The Marshall Mathers LP', 'The Marshall Mathers LP2 (Deluxe)', 'The Marshall Mathers LP2'}
Количество песен: 41
Самый популярный жанр: {'Hip-Hop/Rap'}
Дата релиза альбома: 2013-10-29 12:00:00+00:00
Самая длинная песня: Bad Guy  и её длительность:  0H 7M 14S
Самая короткая песня: Paul Rosenberg (Skit)  и её длительность:  0H 0M 22S
Общая длительность песен: 3H 9M 35S
Средняя длительность песен: 0H 4M 37S
Медиана длительности песен: 0H 4M 48S
Самая дорогая песня: Love the Way You Lie (feat. Rihanna)  и её стоимость:  1.29
Самая дешёвая песня: Without Me  и её стоимость:  -1.0
оОбщая стоимость песен:  52.29
Средняя стоимость песен:  1.28
Медиана стоимости песен:  1.29
Название альбома: {'8 Mile (Music From And Inspired By The Motion Picture)'}
Количество песен: 1
Самый популярный жанр: {'Hip-Hop/Rap'}
Дата релиза альбома: 2002-10-29 08:00:00+00:00
Самая длинная песня: Bad Guy  и её длительность:  0H 7M 14S
Самая короткая песня: Paul Rosenberg (Skit)  и её длительность:

IndexError: list index out of range